## baseline1版本，不参与建模的特征 [ 'os', 'sid’]

In [50]:
## 这个版本主要加了version以及lan特征 

## score1=88.128   加osv
## score2=88.10 加了timestamp  

In [51]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # 不会报warning错误

# 数据加载
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test1.csv')
test = test.iloc[:, 1:]
train = train.iloc[:, 1:]

##### Object类型： lan, os, osv, version, fea_hash
##### 有缺失值的字段： lan, osv

In [52]:
features = train.columns.tolist()
features.remove('label')
print(features)

['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']


In [53]:
#加 osv
train['fea_hash'].map(lambda x: len(str(x))).value_counts()
train['fea1_hash'].map(lambda x: len(str(x))).value_counts()
remove_list = ['os','sid']
col = features
for i in remove_list:
    col.remove(i)
features = train[col]
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features = test[col]
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))


In [54]:
# 定义一个函数 
# 1.去掉'.'和' ' 和'_'
# 2.用众数填充 非数值 如果字符串中有，人能辨识的数值则用该数值填充
# 3. 如果是nan 则众数填充

# error ValueError: invalid literal for int() with base 10: '60十核20G_HD'
#ValueError: invalid literal for int() with base 10: 'Android512'

def app_osv(x):
    x=str(x).replace('Android_','').replace(' ','').replace('.','').replace('60十核20G_HD','').replace('Android','').replace('44W','')
    #
    if x=='nan' or x=='GIONEE_YNGA'or x=='f073b_changxiang_v01_b1b8_20180915' or x==''or x=='%E6%B1%9F%E7%81%B5OS+50':
        return 810
    elif x.count('-')>0:
        return  int(x.split('-')[0])
    else:
        res=int(x)
        if res<10:
            return res*100
        elif res<100:
            return res*10
    return res
            

In [55]:
features['osv']=features['osv'].apply(app_osv)
test_features['osv']=test_features['osv'].apply(app_osv)

In [56]:
# 加timestamp
from datetime import datetime
features['timestamp'] = features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
features['year'] = features['timestamp'].dt.year
features['month'] = features['timestamp'].dt.month
features['day'] = features['timestamp'].dt.day
features['weekday'] = features['timestamp'].dt.weekday
features['hour'] = features['timestamp'].dt.hour
features['minute'] = features['timestamp'].dt.minute
start_time = features['timestamp'].min()
features['timestamp_diff_start'] = features['timestamp']-start_time
features['timestamp_diff_start'] = features['timestamp_diff_start'].dt.days - features['timestamp_diff_start'].dt.seconds/3600/24

In [57]:
test_features['timestamp'] = test_features['timestamp'].apply(lambda x : datetime.fromtimestamp(x/1000))
# 分解时间
test_features['year'] = test_features['timestamp'].dt.year
test_features['month'] = test_features['timestamp'].dt.month
test_features['day'] = test_features['timestamp'].dt.day
test_features['weekday'] = test_features['timestamp'].dt.weekday
test_features['hour'] = test_features['timestamp'].dt.hour
test_features['minute'] = test_features['timestamp'].dt.minute
start_time = test_features['timestamp'].min()
test_features['timestamp_diff_start'] = test_features['timestamp']-start_time
test_features['timestamp_diff_start'] = test_features['timestamp_diff_start'].dt.days - test_features['timestamp_diff_start'].dt.seconds/3600/24

In [58]:
# 加version
# 转化app版本号
def trans_version(version):
    version = version.replace('V','').replace('P_Final_','').replace(' ','').replace('GA','').replace('v','')
    if version == '50':
        return int(5)
    return int(version)
features['version'] = features['version'].apply(trans_version)
test_features['version'] = test_features['version'].apply(trans_version)

In [60]:
# 加 lan 
# 转换lan
def trans_lan(lan):
    lan = str(lan)
    if 'zh' in lan.lower() or 'cn' in lan.lower():
        return int(0)
    else:
        return int(1)
features['lan'] = features['lan'].apply(trans_lan)
test_features['lan']=test_features['lan'].apply(trans_lan)

In [62]:
features


,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,...,cus_type,fea_hash_len,fea1_hash_len,year,month,day,weekday,hour,minute,timestamp_diff_start
0,316361,1199,46000.0,0.0,0.0,0.0,1,104,6.0,900,...,601,10,10,2019,6,7,4,15,32,3.352697
1,135939,893,0.0,0.0,0.0,0.0,1,19,6.0,810,...,1000,10,10,2019,6,8,5,19,40,4.180336
2,399254,821,0.0,760.0,0.0,360.0,1,559,0.0,810,...,696,10,9,2019,6,6,3,23,58,2.001204
3,68983,1004,46000.0,2214.0,0.0,1080.0,1,129,2.0,810,...,753,10,10,2019,6,9,6,8,59,5.625278
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,64,2.0,800,...,582,10,10,2019,6,7,4,8,28,3.646991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1,144,6.0,712,...,373,9,9,2019,6,6,3,23,14,2.031748
499996,346134,1001,0.0,1424.0,0.0,720.0,1,29,2.0,810,...,525,10,10,2019,6,6,3,17,40,2.263287
499997,499635,761,46000.0,1280.0,0.0,720.0,1,54,6.0,601,...,810,10,10,2019,6,5,2,3,14,1.864630
499998,239786,917,46001.0,960.0,0.0,540.0,1,109,2.0,511,...,772,10,10,2019,6,7,4,0,59,3.958727


In [63]:
import lightgbm as lgb
model = lgb.LGBMClassifier() 
model.fit(features.drop(['timestamp' ], axis=1), train['label'])
result = model.predict(test_features.drop(['timestamp'], axis=1))
res = pd.DataFrame(test['sid'])
res['label'] = result
res.to_csv('./baseline_dayone.csv', index=False)
res

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
